In [1]:
!pip install pyarrow
!pip install fastparquet

In [2]:
import numpy as np, pandas as pd, datetime as dt
import matplotlib.pyplot as plt; plt.style.use('ggplot')
import seaborn as sns
from collections import defaultdict

def iter_to_str(iterable):
    return " ".join(map(lambda x: str(0) + str(x), iterable))

def apk(actual, predicted, k=12):
    if len(predicted) > k:
        predicted = predicted[:k]
    score, nhits = 0.0, 0.0
    for i, p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            nhits += 1.0
            score += nhits / (i + 1.0)
    if not actual:
        return 0.0
    return score / min(len(actual), k)

def mapk(actual, predicted, k=12, return_apks=False):
    assert len(actual) == len(predicted)
    apks = [apk(ac, pr, k) for ac, pr in zip(actual, predicted) if 0 < len(ac)]
    if return_apks:
        return apks
    return np.mean(apks)

def blend(dt, w=[], k=12):
    if len(w) == 0:
        w = [1] * (len(dt))
    preds = []
    for i in range(len(w)):
        preds.append(dt[i].split())
    res = {}
    for i in range(len(preds)):
        if w[i] < 0:
            continue
        for n, v in enumerate(preds[i]):
            if v in res:
                res[v] += (w[i] / (n + 1))
            else:
                res[v] = (w[i] / (n + 1))    
    res = list(dict(sorted(res.items(), key=lambda item: -item[1])).keys())
    return ' '.join(res[:k])

def prune(pred, ok_set, k=12):
    pred = pred.split()
    post = []
    for item in pred:
        if int(item) in ok_set and not item in post:
            post.append(item)
    return " ".join(post[:k])

def validation(actual, predicted, grouping, score=0, index=-1, ignore=False, figsize=(12, 6)):
    # actual, predicted : list of lists
    # group : pandas Series
    # score : pandas DataFrame
    
    vc = pd.Series(predicted).apply(len).value_counts()
    print("Fill Rate = ", round(1 - sum(vc[k] * (12 - k) / 12 for k in (set(range(12)) & set(vc.index))) / len(actual), 3) * 100)
    
    
    if ignore: return
    ap12 = mapk(actual, predicted, return_apks=True)
    map12 = round(np.mean(ap12), 6)
    if isinstance(score, int): score = pd.DataFrame({g:[] for g in sorted(grouping.unique().tolist())})
    if index == -1 : index = score.shape[0]
    score.loc[index, "All"] = map12
    plt.figure(figsize=figsize)
    plt.subplot(1, 2, 1); sns.histplot(data=ap12, log_scale=(0, 10), bins=20); plt.title(f"MAP@12 : {map12}")
    for g in grouping.unique():
        map12 = round(mapk(actual[grouping == g], predicted[grouping == g]), 6)
        score.loc[index, g] = map12
    plt.subplot(1, 2, 2); score[[g for g in grouping.unique()[::-1]] + ['All']].loc[index].plot.barh(); plt.title(f"MAP@12 of Groups")
    vc = pd.Series(predicted).apply(len).value_counts()
    score.loc[index, "Fill"] = round(1 - sum(vc[k] * (12 - k) / 12 for k in (set(range(12)) & set(vc.index))) / len(actual), 3) * 100
    display(score)
    return score

In [3]:
sub = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/sample_submission.csv')

In [4]:
sub0 = pd.read_csv("../input/hm-ensemble/submission_uucf0252.csv")
sub['sub0'] = sub0['prediction'].fillna("")
del sub0

In [15]:
sub1 = pd.read_csv("../input/hm-ensemble/submission-blend0249.csv")
sub['sub1'] = sub1['prediction'].fillna("")
del sub1

In [6]:
sub2 = pd.read_csv("../input/hm-ensemble/submission-magic-multi-brend-0240.csv")
sub['sub2'] = sub2['prediction'].fillna("")
del sub2

In [9]:
# sub3 = pd.read_csv("../input/hm-ensemble/submission-mtmt0903-0241.csv")
# sub['sub3'] = sub3['prediction'].fillna("")
# del sub3

In [10]:
# sub4 = pd.read_csv("../input/hm-ensemble/submission_aruaru0239.csv")
# sub['sub4'] = sub4['prediction'].fillna("")
# del sub4

In [7]:
sub.head(3)

,customer_id,prediction,sub0,sub1,sub2
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0706016001 0706016002 0372860001 0610776002 07...,0568601043 0568601006 0918890002 0762846031 09...,0568601043 0568601006 0751471001 0919273002 09...,0568601043 0568601006 0751471001 0924243001 04...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0706016001 0706016002 0372860001 0610776002 07...,0826211002 0448509014 0673677002 0751471001 06...,0826211002 0739590027 0448509014 0706016001 08...,0826211002 0924243001 0924243002 0739590027 08...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0706016001 0706016002 0372860001 0610776002 07...,0794321007 0762846027 0805000001 0794321008 07...,0794321007 0924243001 0852643003 0762846027 07...,0794321007 0852643001 0924243001 0924243002 07...


In [20]:
# 255 252 240 
targets = ['sub0', 'sub1', 'sub2']
weights = [ 1.1, 1.0, 0.6 ] # 0.0252 + 0.0249 + 0.0240

# 0.0255↓
# targets = ['sub0', 'sub1', 'sub2', 'sub3', 'sub4' ]
# weights = [1,1.1,0.8,0.7,0.7]
sub['prediction'] = sub[targets].apply(blend, w=weights, axis=1, k=12)

In [21]:
sub.head(20)

,customer_id,prediction,sub0,sub1,sub2
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0568601043 0568601006 0751471001 0918890002 09...,0568601043 0568601006 0918890002 0762846031 09...,0568601043 0568601006 0751471001 0924243002 05...,0568601043 0568601006 0751471001 0924243001 04...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0826211002 0739590027 0448509014 0673677002 07...,0826211002 0448509014 0673677002 0751471001 06...,0739590027 0826211002 0811835004 0732842008 07...,0826211002 0924243001 0924243002 0739590027 08...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0794321007 0924243001 0852643003 0762846027 08...,0794321007 0762846027 0805000001 0794321008 07...,0794321007 0852643003 0924243001 0852643001 08...,0794321007 0852643001 0924243001 0924243002 07...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,0791587001 0805000001 0448509014 0861803009 08...,0791587001 0861803009 0869331006 0861803014 09...,0791587001 0805000001 0866731001 0732413001 07...,0448509014 0924243002 0924243001 0751471001 05...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,0791587015 0730683050 0927530004 0896152002 07...,0791587015 0927530004 0896152002 0730683050 09...,0730683050 0791587015 0896152002 0751471001 09...,0730683050 0791587015 0927530004 0896152002 09...
5,000064249685c11552da43ef22a5030f35a147f723d5b0...,0791587001 0791587021 0448509014 0905957007 08...,0791587021 0905957007 0448509014 0730683050 07...,0791587001 0852584006 0805000001 0918292001 06...,0448509014 0924243002 0924243001 0573085028 07...
6,0000757967448a6cb83efb3ea7a3fb9d418ac7adf2379d...,0719530003 0448509014 0715624001 0706016001 04...,0719530003 0448509014 0448509001 0719530014 07...,0448509014 0719530003 0715624001 0706016001 07...,0719530003 0448509014 0706016001 0751471001 09...
7,00007d2de826758b65a93dd24ce629ed66842531df6699...,0372860001 0866731001 0160442007 0666444002 01...,0866731001 0372860001 0915529005 0160442010 09...,0372860001 0666444002 0160442007 0909370001 05...,0372860001 0924243002 0924243001 0578478001 01...
8,00007e8d4e54114b5b2a9b51586325a8d0fa74ea23ef77...,0880481001 0715624001 0448509014 0819423001 07...,0880481001 0767423013 0911034001 0874113026 09...,0715624001 0448509014 0706016001 0706016002 07...,0819423001 0448509014 0924243001 0819423002 09...
9,00008469a21b50b3d147c97135e25b4201a8c58997f787...,0448509014 0673677004 0918522001 0924243001 06...,0673677004 0915526001 0673677002 0924243001 09...,0448509014 0918522001 0372860002 0751471001 05...,0448509014 0673677004 0924243001 0573085028 06...


In [ ]:
sub[['customer_id', 'prediction']].to_csv('submission_ensamble.csv', index=False)